In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
from scipy.io import wavfile
import os

In [2]:
def embed_message(audio_path, message, output_path):
    # Ses dosyasını oku
    rate, data = wavfile.read(audio_path)
    if data.ndim > 1:
        data = data[:,0]  # Sadece bir kanal kullan
    data = data.astype(np.float32)

    # Mesajı bit dizisine çevir
    message += '###'  # Mesaj sonu işareti
    message_bits = ''.join(f'{ord(c):08b}' for c in message)
    if len(message_bits) > len(data):
        raise ValueError("Mesaj çok uzun, ses dosyasına sığmıyor.")

    # Mesajı yüksek frekans bileşenlerine göm
    for i, bit in enumerate(message_bits):
        if bit == '1':
            data[i] += 0.005  # Küçük bir değişiklik
        else:
            data[i] -= 0.005

    # Yeni ses dosyasını kaydet
    data = np.clip(data, -1.0, 1.0)
    wavfile.write(output_path, rate, data.astype(np.float32))

def extract_message(audio_path):
    # Ses dosyasını oku
    rate, data = wavfile.read(audio_path)
    if data.ndim > 1:
        data = data[:,0]
    data = data.astype(np.float32)

    # Mesajı çıkar
    bits = ''
    for i in range(len(data)):
        if data[i] > 0:
            bits += '1'
        else:
            bits += '0'
        if bits.endswith('001000110010001100100011'):  # '###' bit dizisi
            message = ''.join(chr(int(bits[i:i+8], 2)) for i in range(0, len(bits)-24, 8))
            return message
    return ''

In [3]:
def select_audio():
    path = filedialog.askopenfilename(filetypes=[("WAV files", "*.wav")])
    entry_audio.delete(0, tk.END)
    entry_audio.insert(0, path)

def save_audio():
    path = filedialog.asksaveasfilename(defaultextension=".wav", filetypes=[("WAV files", "*.wav")])
    entry_output.delete(0, tk.END)
    entry_output.insert(0, path)

def hide_message():
    audio_path = entry_audio.get()
    output_path = entry_output.get()
    message = entry_message.get()
    if len(message) > 160:
        messagebox.showerror("Hata", "Mesaj 160 karakterden uzun olamaz.")
        return
    try:
        embed_message(audio_path, message, output_path)
        messagebox.showinfo("Başarılı", "Mesaj başarıyla gizlendi.")
    except Exception as e:
        messagebox.showerror("Hata", str(e))

def reveal_message():
    audio_path = entry_audio.get()
    try:
        message = extract_message(audio_path)
        messagebox.showinfo("Gizli Mesaj", message)
    except Exception as e:
        messagebox.showerror("Hata", str(e))

In [4]:


# Arayüz oluştur
root = tk.Tk()
root.title("Maskeleme ve Filtreleme Steganografi")

tk.Label(root, text="Giriş Ses Dosyası:").grid(row=0, column=0, sticky="e")
entry_audio = tk.Entry(root, width=50)
entry_audio.grid(row=0, column=1)
tk.Button(root, text="Seç", command=select_audio).grid(row=0, column=2)

tk.Label(root, text="Çıkış Ses Dosyası:").grid(row=1, column=0, sticky="e")
entry_output = tk.Entry(root, width=50)
entry_output.grid(row=1, column=1)
tk.Button(root, text="Kaydet", command=save_audio).grid(row=1, column=2)

tk.Label(root, text="Mesaj:").grid(row=2, column=0, sticky="e")
entry_message = tk.Entry(root, width=50)
entry_message.grid(row=2, column=1, columnspan=2)

tk.Button(root, text="Mesajı Gizle", command=hide_message).grid(row=3, column=1)
tk.Button(root, text="Mesajı Göster", command=reveal_message).grid(row=3, column=2)

root.mainloop()